# Joins en Spark

## Requisitos Previos

Instalar Spark y Java en la Máquina Virtual (VM)

In [1]:
# instalar Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# descargar spark 3.5.0
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [2]:
ls -l # verificar que el .tgz está ahí

total 391016
drwxr-xr-x 1 root root      4096 Nov 17 14:29 sample_data/
-rw-r--r-- 1 root root 400395283 Sep  9  2023 spark-3.5.0-bin-hadoop3.tgz


In [3]:
# descomprimirlo
!tar xf spark-3.5.0-bin-hadoop3.tgz

In [4]:
!pip install -q findspark

In [5]:

!pip install py4j

# Para maps
!pip install folium
!pip install plotly

Definir el entorno

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Iniciar Sesión de Spark (Spark Session)

---

In [7]:
import findspark
findspark.init("spark-3.5.0-bin-hadoop3")# SPARK_HOME

from pyspark.sql import SparkSession

# crear la sesión
spark = SparkSession \
        .builder \
        .appName("DataFrames Basics") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.5.0'

In [8]:
spark

In [9]:
# Para optimización de conversión a Pandas
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [10]:
# Importar funciones sql
from pyspark.sql.functions import *

Descargar y subir los conjuntos de datos (datasets) (GUITARS.JSON, GUITARPLAYERS.JSON, BANDS.JSON)

In [11]:
from google.colab import files
uploaded = files.upload()

Saving guitarPlayers.json to guitarPlayers.json
Saving guitars.json to guitars.json
Saving bands.json to bands.json


In [12]:
tiposGuitarrasDF = spark.read \
    .option("inferSchema", "true") \
    .json("guitars.json")

guitarristasDF = spark.read \
    .option("inferSchema", "true") \
    .json("guitarPlayers.json")

bandasDF = spark.read \
    .option("inferSchema", "true") \
    .json("bands.json")

In [13]:
tiposGuitarrasDF.show(3)
guitarristasDF.show(3)
bandasDF.show(3)

+--------------------+---+------+------------+
|          guitarType| id|  make|       model|
+--------------------+---+------+------------+
|Electric double-n...|  0|Gibson|    EDS-1275|
|            Electric|  5|Fender|Stratocaster|
|            Electric|  1|Gibson|          SG|
+--------------------+---+------+------------+
only showing top 3 rows

+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   0|    [0]|  0|  Jimmy Page|
|   1|    [1]|  1| Angus Young|
|   2| [1, 5]|  2|Eric Clapton|
+----+-------+---+------------+
only showing top 3 rows

+-----------+---+------------+----+
|   hometown| id|        name|year|
+-----------+---+------------+----+
|     Sydney|  1|       AC/DC|1973|
|     London|  0|Led Zeppelin|1968|
|Los Angeles|  3|   Metallica|1981|
+-----------+---+------------+----+
only showing top 3 rows



## Ejemplos

Inner Join =  todo lo que hay en AMBOS DF y se selecciona la columna que es común en los dos

In [14]:
joinCondition = guitarristasDF.band == bandasDF.id
guitarristasconbandasDF = guitarristasDF.join(bandasDF, joinCondition, "inner")
guitarristasconbandasDF.show(3)

+----+-------+---+------------+-----------+---+------------+----+
|band|guitars| id|        name|   hometown| id|        name|year|
+----+-------+---+------------+-----------+---+------------+----+
|   1|    [1]|  1| Angus Young|     Sydney|  1|       AC/DC|1973|
|   0|    [0]|  0|  Jimmy Page|     London|  0|Led Zeppelin|1968|
|   3|    [3]|  3|Kirk Hammett|Los Angeles|  3|   Metallica|1981|
+----+-------+---+------------+-----------+---+------------+----+



In [15]:
# guitarristasconbandasDF.select("id", "band").show(3) # fallará porque hay dos "id"

In [16]:
# you can rename the column
bandasModDF = bandasDF.withColumnRenamed("id", "bandId") \
    .withColumnRenamed("name", "bandName")
guitarristasconbandasDF2 = guitarristasDF.join(bandasModDF,
    guitarristasDF.band == bandasModDF.bandId, "inner").orderBy("name")
guitarristasconbandasDF2.show(3)

# Hay mas opciones, pero esta es la "best practice"

+----+-------+---+------------+-----------+------+------------+----+
|band|guitars| id|        name|   hometown|bandId|    bandName|year|
+----+-------+---+------------+-----------+------+------------+----+
|   1|    [1]|  1| Angus Young|     Sydney|     1|       AC/DC|1973|
|   0|    [0]|  0|  Jimmy Page|     London|     0|Led Zeppelin|1968|
|   3|    [3]|  3|Kirk Hammett|Los Angeles|     3|   Metallica|1981|
+----+-------+---+------------+-----------+------+------------+----+



Left Outer = todo lo que tenemos con el inner sumado a todo lo que tenga el DF de la izquierda, y si no existe ese registro en el de la derecha, se rellena con nulos

In [17]:
joinCondition2 = guitarristasDF.band == bandasModDF.bandId
guitarristasDF.join(bandasModDF, joinCondition2, "left_outer").orderBy("name").show(3)
# nulos en Eric Calpton

+----+-------+---+------------+--------+------+------------+----+
|band|guitars| id|        name|hometown|bandId|    bandName|year|
+----+-------+---+------------+--------+------+------------+----+
|   1|    [1]|  1| Angus Young|  Sydney|     1|       AC/DC|1973|
|   2| [1, 5]|  2|Eric Clapton|    NULL|  NULL|        NULL|NULL|
|   0|    [0]|  0|  Jimmy Page|  London|     0|Led Zeppelin|1968|
+----+-------+---+------------+--------+------+------------+----+
only showing top 3 rows



  Right Outer = todo lo que tiene el inner sumado a todo lo que tenga el DF de la derecha, y si no existe ese registro en el de la izquiera, se rellena con nulos (AL CONTRARIO QUE EL DE ARRIBA)


In [18]:
guitarristasDF.join(bandasModDF, joinCondition2, "right_outer").orderBy("name").show(3)
# nulos en los Beatles

+----+-------+----+-----------+--------+------+-----------+----+
|band|guitars|  id|       name|hometown|bandId|   bandName|year|
+----+-------+----+-----------+--------+------+-----------+----+
|NULL|   NULL|NULL|       NULL|New York|     4|The Ramones|1974|
|NULL|   NULL|NULL|       NULL|  Dublin|     5|         U2|1976|
|   1|    [1]|   1|Angus Young|  Sydney|     1|      AC/DC|1973|
+----+-------+----+-----------+--------+------+-----------+----+
only showing top 3 rows



 Outer Join = todo lo que tiene el inner sumado a todo lo que tienen ambos DF por su parte, rellenando como en los aneteriores los registros que no existan con nulos

In [19]:
guitarristasDF.join(bandasModDF, joinCondition2, "outer").orderBy("name").show(3)
# nulos en ambos ahora

+----+-------+----+-----------+--------+------+-----------+----+
|band|guitars|  id|       name|hometown|bandId|   bandName|year|
+----+-------+----+-----------+--------+------+-----------+----+
|NULL|   NULL|NULL|       NULL|New York|     4|The Ramones|1974|
|NULL|   NULL|NULL|       NULL|  Dublin|     5|         U2|1976|
|   1|    [1]|   1|Angus Young|  Sydney|     1|      AC/DC|1973|
+----+-------+----+-----------+--------+------+-----------+----+
only showing top 3 rows



  Semi-joins/ Left Semi Join = Devuelve las filas del DF de la izquierda que tengan match con el de la derecha, pero no trae nada del de la derecha

In [20]:
guitarristasDF.join(bandasModDF, joinCondition2, "left_semi").orderBy("name").show(3)


+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   1|    [1]|  1| Angus Young|
|   0|    [0]|  0|  Jimmy Page|
|   3|    [3]|  3|Kirk Hammett|
+----+-------+---+------------+



Anti-joins = Devuelve las filas del DF de la izquierda que NO tienen match con el de la derecha, no devuelve este nada tampoco del de la derecha


In [21]:
guitarristasDF.join(bandasModDF, joinCondition2, "anti").orderBy("name").show(3)

+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   2| [1, 5]|  2|Eric Clapton|
+----+-------+---+------------+



Cross-join = combina cada fila del DF de la izquierda con cada fila del DF de la derecha (no muy usado)

In [22]:
guitarristasDF.crossJoin(bandasModDF).show(20)

+----+-------+---+------------+-----------+------+------------+----+
|band|guitars| id|        name|   hometown|bandId|    bandName|year|
+----+-------+---+------------+-----------+------+------------+----+
|   0|    [0]|  0|  Jimmy Page|     Sydney|     1|       AC/DC|1973|
|   1|    [1]|  1| Angus Young|     Sydney|     1|       AC/DC|1973|
|   2| [1, 5]|  2|Eric Clapton|     Sydney|     1|       AC/DC|1973|
|   3|    [3]|  3|Kirk Hammett|     Sydney|     1|       AC/DC|1973|
|   0|    [0]|  0|  Jimmy Page|     London|     0|Led Zeppelin|1968|
|   1|    [1]|  1| Angus Young|     London|     0|Led Zeppelin|1968|
|   2| [1, 5]|  2|Eric Clapton|     London|     0|Led Zeppelin|1968|
|   3|    [3]|  3|Kirk Hammett|     London|     0|Led Zeppelin|1968|
|   0|    [0]|  0|  Jimmy Page|Los Angeles|     3|   Metallica|1981|
|   1|    [1]|  1| Angus Young|Los Angeles|     3|   Metallica|1981|
|   2| [1, 5]|  2|Eric Clapton|Los Angeles|     3|   Metallica|1981|
|   3|    [3]|  3|Kirk Hammett|Los

*Union Join = Concatena dos DF pero solo si tienen el mismo esquema (aqui no es el caso)

## Ejercicios
1. Lee los CSVs customers.csv, orders.csv, products.csv y regions.csv en DataFrames de Spark.

2. Muestra todos los clientes y su pedido más caro (ten en cuenta que un cliente puede tener varios pedidos registrados).

3. Muestra todos los clientes que nunca han hecho un pedido. Compruébalo obteniendo todos los clientes con pedidos y verificando que no estén en la tabla.

4. Encuentra los productos comprados por los 10 clientes que más dinero han gastado en total (ten en cuenta que un cliente puede haber comprado varios productos distintos).

Ejercicio 1

Ejercicio 2

Ejercicio 3

Ejercicio 4